# 🛠️ Notebook 02: Tool Use and Agentic Reasoning with the Responses API

This lab shows how to register Python utilities as **tools** so the Responses API can plan, call, and explain its work.

## What you'll build
- a lightweight calculator and repo file reader
- a tool registry the model can invoke
- a streaming tracer to watch planner/executor events

## Prerequisites
1. `OPENAI_API_KEY` set in your `.env` (loaded automatically).
2. `pip install -r requirements.txt`.
3. Run this notebook from the repo root so file paths resolve.

In [ ]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Any, Dict, List

from openai import OpenAI

from utils.openai_client import get_response

client = OpenAI()
REPO_ROOT = Path('.')

## 1. Define Python tools the model can call
We'll keep it simple: a calculator for basic arithmetic and a file reader for pulling short excerpts from this repository.

In [ ]:
def basic_calculator(operation: str, a: float, b: float) -> str:
    """Deterministic math helper that supports +, -, *, and /."""
    if operation not in {"add", "subtract", "multiply", "divide"}:
        return f"Unsupported operation: {operation}"

    if operation == "add":
        value = a + b
    elif operation == "subtract":
        value = a - b
    elif operation == "multiply":
        value = a * b
    else:
        if b == 0:
            return "Cannot divide by zero."
        value = a / b

    return f"{value:.4f}"


MAX_FILE_CHARS = 600


def read_repo_file(relative_path: str) -> str:
    """Return a trimmed excerpt from a text file inside the repo."""
    repo_root = REPO_ROOT.resolve()
    target = (repo_root / relative_path).resolve()

    if not target.is_relative_to(repo_root):
        return "Only files within this repository can be read."

    try:
        text = target.read_text(encoding="utf-8")
    except FileNotFoundError:
        return f"File {relative_path} was not found."
    except UnicodeDecodeError:
        return f"File {relative_path} is not plain text."

    return text[:MAX_FILE_CHARS]


In [ ]:
TOOLS: List[Dict[str, Any]] = [
    {
        'type': 'function',
        'function': {
            'name': 'basic_calculator',
            'description': 'Perform deterministic arithmetic (add, subtract, multiply, divide).',
            'parameters': {
                'type': 'object',
                'properties': {
                    'operation': {
                        'type': 'string',
                        'enum': ['add', 'subtract', 'multiply', 'divide'],
                        'description': 'Math operation to execute.',
                    },
                    'a': {
                        'type': 'number',
                        'description': 'First operand.',
                    },
                    'b': {
                        'type': 'number',
                        'description': 'Second operand.',
                    },
                },
                'required': ['operation', 'a', 'b'],
            },
        },
    },
    {
        'type': 'function',
        'function': {
            'name': 'read_repo_file',
            'description': 'Read a short excerpt from a text file in this repository.',
            'parameters': {
                'type': 'object',
                'properties': {
                    'relative_path': {
                        'type': 'string',
                        'description': 'Path relative to the repo root (e.g., README.md).',
                    }
                },
                'required': ['relative_path'],
            },
        },
    },
]

PYTHON_TOOL_REGISTRY = {
    'basic_calculator': basic_calculator,
    'read_repo_file': read_repo_file,
}

## 2. Register the tools in a helper call
Because `get_response` forwards `tools=` to the SDK, we can reuse the helper from Notebook 01 and immediately unlock richer behaviors.

In [ ]:
input_messages = [
    {
        'role': 'system',
        'content': [{'type': 'text', 'text': 'You are a helpful planning assistant. Use tools when necessary and explain your steps.'}],
    },
    {
        'role': 'user',
        'content': [{'type': 'text', 'text': "You can reason step by step. If you need repository context, call the file reader tool; for arithmetic, use the calculator. What is the difference between 2024 and 1998, and summarize the README intro in one sentence?"}],
    },
]

response_text = get_response(
    input_messages,
    model='gpt-4.1-mini',
    tools=TOOLS,
    reasoning={'effort': 'medium'},
)

print(response_text)

## 3. Watch the planner/executor trace with streaming events
The SDK's streaming interface emits events whenever the model reasons, plans, or calls a tool. Capturing those events turns the call into a narrated trace.

In [ ]:
def run_with_trace(question: str) -> None:
    stream = client.responses.stream(
        model='gpt-4.1-mini',
        input=[
            {
                'role': 'system',
                'content': [
                    {
                        'type': 'text',
                        'text': 'Be a careful planner. Think out loud, call tools when needed, and cite the tool outputs before answering.',
                    }
                ],
            },
            {
                'role': 'user',
                'content': [{'type': 'text', 'text': question}],
            },
        ],
        tools=TOOLS,
        reasoning={'effort': 'medium'},
    )

    with stream as events:
        for event in events:
            if event.type == 'response.output_text.delta':
                print(f"🧠 model reasoning: {event.delta}")
            elif event.type == 'response.tool_call.delta':
                delta = event.delta
                name = delta.get('name', 'tool')
                args = delta.get('arguments')
                print(f"🔧 planning tool call -> {name}({args})")
            elif event.type == 'response.tool_call.completed':
                call = event.tool_call
                fn_name = call.function.name
                parsed_args = json.loads(call.function.arguments or '{}')
                tool_result = PYTHON_TOOL_REGISTRY.get(fn_name, lambda **_: 'Tool not found.')(**parsed_args)
                print(f"✅ executed {fn_name} with {parsed_args} => {tool_result}")
                events.send(
                    {
                        'type': 'response.tool_output',
                        'tool_call_id': call.id,
                        'output': tool_result,
                    }
                )
            elif event.type == 'response.completed':
                final = events.get_final_response()
                text_blocks = []
                for item in getattr(final, 'output', []):
                    if getattr(item, 'type', '') == 'message':
                        for content in getattr(item, 'content', []):
                            if getattr(content, 'type', '') == 'output_text':
                                text_blocks.append(content.text)
                if text_blocks:
                    print("
🎉 Final answer:
" + '
'.join(text_blocks))
                break

run_with_trace('Use the calculator to compute 13.5 * 1.2 and quote a single sentence from README.md that mentions the project goals.')

## 4. Reflection questions
1. How would you add a tool that performs live web searches?
2. What safeguards should you implement before letting the model read arbitrary files?
3. Try swapping `gpt-4.1-mini` for a cheaper/faster model—does the planner still pick the right tools?